In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader

In [3]:
torch.cuda.is_available()

True

In [5]:
bat_dict = pickle.load(open(r'./Data/bat_dict.pkl', 'rb'))

In [6]:
print(bat_dict.keys())
print(bat_dict["b1c0"].keys())
print(bat_dict["b1c0"]["summary"].keys())

dict_keys(['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4', 'b1c5', 'b1c6', 'b1c7', 'b1c9', 'b1c11', 'b1c14', 'b1c15', 'b1c16', 'b1c17', 'b1c18', 'b1c19', 'b1c20', 'b1c21', 'b1c23', 'b1c24', 'b1c25', 'b1c26', 'b1c27', 'b1c28', 'b1c29', 'b1c30', 'b1c31', 'b1c32', 'b1c33', 'b1c34', 'b1c35', 'b1c36', 'b1c37', 'b1c38', 'b1c39', 'b1c40', 'b1c41', 'b1c42', 'b1c43', 'b1c44', 'b1c45', 'b2c0', 'b2c1', 'b2c2', 'b2c3', 'b2c4', 'b2c5', 'b2c6', 'b2c10', 'b2c11', 'b2c12', 'b2c13', 'b2c14', 'b2c17', 'b2c18', 'b2c19', 'b2c20', 'b2c21', 'b2c22', 'b2c23', 'b2c24', 'b2c25', 'b2c26', 'b2c27', 'b2c28', 'b2c29', 'b2c30', 'b2c31', 'b2c32', 'b2c33', 'b2c34', 'b2c35', 'b2c36', 'b2c37', 'b2c38', 'b2c39', 'b2c40', 'b2c41', 'b2c42', 'b2c43', 'b2c44', 'b2c45', 'b2c46', 'b2c47'])
dict_keys(['cycle_life', 'charge_policy', 'summary', 'cycles'])
dict_keys(['IR', 'QC', 'QD', 'Tavg', 'Tmin', 'Tmax', 'chargetime', 'cycle'])


In [7]:
b1 = []
b2 = []

for bat in bat_dict.keys():
    if bat_dict[bat]["summary"]["QD"][0] < 0.9:
        b1.append(bat)
    
    for i in range(1, len(bat_dict[bat]["summary"]["QD"])):
        if bat_dict[bat]["summary"]["QD"][i] > 1.1:
            b2.append(bat)

print(b1)
print(b2)

['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4', 'b1c5', 'b1c6', 'b1c7', 'b1c9', 'b1c11', 'b1c14', 'b1c15', 'b1c16', 'b1c17', 'b1c18', 'b1c19', 'b1c20', 'b1c21', 'b1c23', 'b1c24', 'b1c25', 'b1c26', 'b1c27', 'b1c28', 'b1c29', 'b1c30', 'b1c31', 'b1c32', 'b1c33', 'b1c34', 'b1c35', 'b1c36', 'b1c37', 'b1c38', 'b1c39', 'b1c40', 'b1c41', 'b1c42', 'b1c43', 'b1c44', 'b1c45']
['b1c0', 'b1c18', 'b2c12', 'b2c44']


In [8]:
max_cycle = -np.inf
min_cycle = np.inf

for bat in bat_dict.keys():
    if bat_dict[bat]["cycle_life"] > max_cycle:
        max_cycle = bat_dict[bat]["cycle_life"][0][0]
    
    if bat_dict[bat]["cycle_life"] < min_cycle:
        min_cycle = bat_dict[bat]["cycle_life"][0][0]

print(max_cycle, min_cycle)

2237.0 148.0


In [9]:
for bat in bat_dict.keys():
    if bat in b1:
        for key in bat_dict[bat]["summary"].keys():
            bat_dict[bat]["summary"][key] = bat_dict[bat]["summary"][key][1:]

In [ ]:
class BatteryDataset(Dataset):
    def __init__(self, bat_dict, features, target):
        self.seqs = []
        for key, rec in bat_dict.items():
            data = rec["summary"]
            seq = torch.tensor([[d[f] for f in features] for d in data], dtype=torch.float)
            self.seqs.append(seq)
        self.target = target
    
    def __len__(self):
        return len(self.seqs)
    
    def __getitem__(self, idx):
        seq = self.seqs[idx]
        return seq[:-1], seq[1:] #input is all but last, target is all but first

In [11]:
def collate_fn(batch):
    # batch is list of (inp_seq, target_seq)
    inputs, targets = zip(*batch)
    lengths = torch.tensor([s.size(0) for s in inputs])
    inputs_padded = pad_sequence(inputs, batch_first=True)
    targets_padded = pad_sequence(targets, batch_first=True)

    #pack for rnn
    packed_inputs = pack_padded_sequence(inputs_padded, lengths, batch_first=True, enforce_sorted=False)
    return packed_inputs, targets_padded, lengths

In [ ]:
q_nominal = 1.1
features = ["cycle", "IR", "QC", "Tavg", "Tmin", "Tmax", "chargetime"]
target = ["QD"]